In [ ]:
#Import dependencies
import csv
import pandas as pd
import matplotlib.pyplot as plt
import gmaps
import gmaps.datasets
import requests
import json
import seaborn as sns
from config import gkey 
from random import sample

In [ ]:
#Load CSV
accident_data = pd.read_csv("Resources/2021_us_accidents.csv")
accident_data.head()

In [ ]:
#Add a column to count each accident as 1
accident_data["# of accidents"] = 1
accident_data.head()

In [ ]:
#See how many rows there are
len(accident_data["State"])

In [ ]:
#Pull out columns for the heatmap
accident_data[["Start_Lat","Start_Lng","State","Severity"]][724:737]

In [ ]:
#Create a sample for the limitations of our API key
sample = accident_data.sample(n = 10 , replace = False)
print(sample)

In [ ]:
#Data for just Minnesota
MN_accident_data = accident_data.loc[accident_data["State"] == "MN"]


In [ ]:
ustates = []
for x in accident_data["State"]:
    if x not in ustates:
        ustates.append(x)
print(ustates)
#This list includes the continential US and Washington DC. (Excludes AK and HI)

In [ ]:
#Scatter plot of accidents in our data
# x_values = accident_data["Start_Lng"]
# y_values = accident_data["Start_Lat"]
# fig, ac = plt.subplots(figsize = (10,7),)
# ac.scatter(x_values,y_values, facecolors = "r", edgecolors = "b", alpha = 0.5)
# plt.xlabel("Longitude")
# plt.ylabel("Latitude")
# plt.show()

fig, ax = plt.subplots(figsize=(15.7,10.27))
_severity= ['1', '2', '3', '4']


sns.scatterplot(data=accident_data, x="Start_Lng", y="Start_Lat", hue='Severity', palette=['green','blue'])
plt.grid()
plt.title("Accidents in US by Severity")
plt.xlabel("Longitude")
plt.ylabel("Latitude")

In [ ]:
#Code for heatmap
# gmaps_key = gmaps.configure(api_key = gkey)
# locations = sample[["Start_Lat","Start_Lng"]]
# weights = sample["# of accidents"][0:10]

In [ ]:
#Code to display heatmap
# fig = gmaps.figure()
# fig.add_layer(gmaps.heatmap_layer(locations, weights = weights[0:10]))
# fig

In [ ]:
accbystate = accident_data[["State", "Severity","# of accidents"]]
accbystate.groupby(['State', 'Severity']).size().unstack().plot(kind='barh', stacked=True, figsize = (15, 11))
plt.title("Accidents and Severity by State")
plt.xlabel("Number of Accidents")
plt.tight_layout

In [ ]:
groupby = accident_data.groupby(['Visibility(mi)'])['# of accidents']
groupby.head()

In [ ]:
vis_data = accident_data.groupby("Visibility(mi)").count()
vis_data["# of accidents"]

In [ ]:
vis_data.reset_index(inplace = True)
vis_data

In [ ]:
#Scatter plot of visibility vs accidents
groupby = accident_data.groupby(['Visibility(mi)'])['# of accidents'].sum()
x_values = vis_data["Visibility(mi)"]
y_values = vis_data["# of accidents"]
plt.scatter(x_values,y_values)
plt.xlabel("Visibility(mi)")
plt.ylabel("# of accidents")
plt.show()

In [ ]:
daynight = accident_data[["Sunrise_Sunset","# of accidents"]].groupby("Sunrise_Sunset").count()
daynight.plot.pie(y = "# of accidents", legend = False, autopct = "%1.2f%%")

In [ ]:
roundabouts = (accident_data.loc[accident_data["Roundabout"] == True]).sum()
stop = (accident_data.loc[accident_data["Stop"] == True]).sum()
signal = (accident_data.loc[accident_data["Traffic_Signal"] == True]).sum()
crosswalk = (accident_data.loc[accident_data["Crossing"] == True]).sum()
none = accident_data.loc[(accident_data["Roundabout"] == False) & (accident_data["Stop"] == False) & (accident_data["Traffic_Signal"] == False) & (accident_data["Crossing"] == False)].sum()

In [ ]:
none["# of accidents"]

In [ ]:
crosswalk["Crossing"]

In [ ]:
distruptions = [roundabouts["Roundabout"], stop["Stop"], signal["Traffic_Signal"], crosswalk["Crossing"], none["# of accidents"]]
distruption_labels = ["Roundabouts", "Stop Sign", "Traffic Signal", "Crosswalk", "No Obstruction"]
colors = ["yellow", "red", "green", "purple", "blue"]
explode = (0,0,0,0,0.3)
plt.pie(distruptions, explode=explode, labels=distruption_labels, colors=colors, autopct="%1.1f%%", shadow=True)